In [ ]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
import sklearn as skl
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações de visualização
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Bibliotecas carregadas com sucesso")

# 1. Carregamento dos dados (certifique-se que os arquivos estejam no caminho correto)
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")

print("Dados de treino carregados com sucesso. Shape: {}".format(train_df.shape))
print("Dados de teste carregados com sucesso. Shape: {}".format(test_df.shape))

print("\nPrimeiras 5 linhas dos dados de treino:")
print(train_df.head())

# 2. Análise Exploratória de Dados (EDA) Inicial
print("\nInformações gerais sobre os dados de treino:")
train_df.info()

print("\nEstatísticas descritivas dos dados de treino:")
print(train_df.describe())

print("\nValores ausentes nos dados de treino:")
print(train_df.isnull().sum()[train_df.isnull().sum() > 0])

print("\nDistribuição da variável alvo (labels):")
print(train_df["labels"].value_counts())
print(train_df["labels"].value_counts(normalize=True) * 100)

# 3. Tratamento de Valores Ausentes
cols_to_impute_median = [
    'age_first_funding_year', 'age_last_funding_year',
    'age_first_milestone_year', 'age_last_milestone_year',
    'funding_total_usd'
]

for col in cols_to_impute_median:
    median_val = train_df[col].median()
    train_df[col].fillna(median_val, inplace=True)
    test_df[col].fillna(median_val, inplace=True)

print("Valores ausentes tratados por mediana para as colunas: {}.".format(cols_to_impute_median))
print("\nVerificando valores ausentes após imputação nos dados de treino:")
print(train_df[cols_to_impute_median].isnull().sum())

# 4. One-Hot Encoding para category_code
categorical_col = ["category_code"]

train_df = pd.get_dummies(train_df, columns=categorical_col, drop_first=True)
test_df = pd.get_dummies(test_df, columns=categorical_col, drop_first=True)

# Alinhar colunas de treino e teste (evita problemas quando categorias não aparecem no teste)
missing_cols_in_test = set(train_df.columns) - set(test_df.columns)
missing_cols_in_train = set(test_df.columns) - set(train_df.columns)

for col in missing_cols_in_test:
    if col != 'labels':  # Não adicionar a coluna alvo no teste
        test_df[col] = 0

for col in missing_cols_in_train:
    if col != 'labels':  # Não adicionar a coluna alvo no treino
        train_df[col] = 0

# Reordenar as colunas do teste para ficar igual ao treino
test_df = test_df[train_df.drop('labels', axis=1).columns]

print("One-Hot Encoding aplicado para a coluna {}.".format(categorical_col[0]))
print("\nNovas dimensões dos dados de treino após encoding: {}".format(train_df.shape))
print("Novas dimensões dos dados de teste após encoding: {}".format(test_df.shape))

print("\nPrimeiras 5 linhas dos dados de treino após One-Hot Encoding:")
print(train_df.head())

# 5. Separação de Features e Target, e Padronização
from sklearn.preprocessing import StandardScaler

X_train = train_df.drop("labels", axis=1)
y_train = train_df["labels"]

X_test = test_df.copy()

numerical_cols = [
    col for col in X_train.columns
    if X_train[col].dtype in ["int64", "float64"]
    and not col.startswith("is_")
    and not col.startswith("category_code_")
]

scaler = StandardScaler()

X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

print("Features e target separados. Colunas numéricas padronizadas.")
print("\nPrimeiras 5 linhas de X_train após padronização:")
print(X_train.head())

# 6. Treinamento e Avaliação do Modelo (Regressão Logística)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

model = LogisticRegression(random_state=42, solver='liblinear')
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)

print("Modelo de Regressão Logística treinado com sucesso!")
print("\nRelatório de Classificação no conjunto de treino:")
print(classification_report(y_train, y_pred_train))
print("Acurácia no conjunto de treino: {:.2f}".format(accuracy_score(y_train, y_pred_train)))

predictions = model.predict(X_test)
print("Previsões geradas para o conjunto de teste.")
print("\nPrimeiras 5 previsões para o conjunto de teste:")
print(predictions[:5])

# 7. Geração do Arquivo de Submissão para o Kaggle
submission_df = pd.DataFrame({
    "id": test_df["id"],  # Certifique-se que há uma coluna "id" no test_df
    "labels": predictions
})

submission_df.to_csv("../data/submission.csv", index=False)

print("Arquivo de submissão gerado com sucesso: ../data/submission.csv")
print("\nPrimeiras 5 linhas do arquivo de submissão:")
print(submission_df.head())